In [ ]:
'''
BSs (LTE’s eNodeBs) can be
either in an active state (handling UEs’ data) or in an idle state
(transmitting only downlink control signaling).

the classifier determines its
class for the next period of time. We defined the following
two classes:
 IDLE
 DOWNLOADING

CLASSIFICATION: Random Forest, K means clustering

'''

from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
import pandas as pd

df =pd.read_csv("drive/My Drive/Emerged.csv")
df.head()

,Timestamp,Longitude,Latitude,Speed,Operatorname,CellID,NetworkMode,RSRP,RSRQ,SNR,CQI,RSSI,DL_bitrate,UL_bitrate,State,NRxRSRP,NRxRSRQ,ServingCell_Lon,ServingCell_Lat,ServingCell_Distance
0,2017.11.21_15.03.50,-8.499701,51.893336,0,A,2,LTE,-95,-13,4.0,10,-80,0,0,D,-,-,-8.491719,51.893905,551.37
1,2017.11.21_15.03.51,-8.499701,51.893336,0,A,2,LTE,-95,-13,2.0,8,-78,0,0,I,-,-,-8.491719,51.893905,551.37
2,2017.11.21_15.03.52,-8.499733,51.893346,0,A,2,LTE,-95,-13,13.0,9,-80,0,0,I,-,-,-8.491719,51.893905,553.42999999999995
3,2017.11.21_15.03.52,-8.499733,51.893346,0,A,2,LTE,-95,-13,13.0,9,-80,0,0,I,-,-,-8.491719,51.893905,553.42999999999995
4,2017.11.21_15.03.53,-8.499887,51.893384,1,A,2,LTE,-95,-13,13.0,9,-80,0,0,I,-,-,-8.491719,51.893905,563.48000000000002


In [ ]:
df.shape

(174658, 20)

In [ ]:
print(df.columns.values)

['Timestamp' 'Longitude' 'Latitude' 'Speed' 'Operatorname' 'CellID'
 'NetworkMode' 'RSRP' 'RSRQ' 'SNR' 'CQI' 'RSSI' 'DL_bitrate' 'UL_bitrate'
 'State' 'NRxRSRP' 'NRxRSRQ' 'ServingCell_Lon' 'ServingCell_Lat'
 'ServingCell_Distance']


In [ ]:
for i in df.columns:

  if df[i].isnull().sum()>0:
    df[i].fillna(df[i].mode()[0],inplace=True)

In [ ]:
df.isnull().sum()

Timestamp               0
Longitude               0
Latitude                0
Speed                   0
Operatorname            0
CellID                  0
NetworkMode             0
RSRP                    0
RSRQ                    0
SNR                     0
CQI                     0
RSSI                    0
DL_bitrate              0
UL_bitrate              0
State                   0
NRxRSRP                 0
NRxRSRQ                 0
ServingCell_Lon         0
ServingCell_Lat         0
ServingCell_Distance    0
dtype: int64

In [ ]:
df.drop(df.index[(df["Longitude"] == "Longitude")],axis=0,inplace=True)    #REMOVING string VALUES

In [ ]:
df.shape

(174524, 20)

In [ ]:
df=df.drop(['Operatorname','NetworkMode','NRxRSRP','NRxRSRQ','Timestamp'], axis=1)

In [ ]:
df.dtypes                     #rsrq,snr,rssi,cqi

Longitude               object
Latitude                object
Speed                   object
CellID                  object
RSRP                    object
RSRQ                    object
SNR                     object
CQI                     object
RSSI                    object
DL_bitrate              object
UL_bitrate              object
State                   object
ServingCell_Lon         object
ServingCell_Lat         object
ServingCell_Distance    object
dtype: object

In [ ]:

df['ServingCell_Distance'] = df['ServingCell_Distance'].replace(['-'], 0)

In [ ]:
df['ServingCell_Distance'] = df['ServingCell_Distance'].replace(0,df["ServingCell_Distance"].mode()[0])

In [ ]:
df["ServingCell_Distance"] = df["ServingCell_Distance"].apply(pd.to_numeric)


In [ ]:
df['ServingCell_Lon'] = df['ServingCell_Lon'].replace(['-'], 0)
df['ServingCell_Lat'] = df['ServingCell_Lat'].replace(['-'], 0)
df['RSRQ'] = df['RSRQ'].replace(['-'], 0)
df['SNR'] = df['SNR'].replace(['-'], 0)
df['CQI'] = df['CQI'].replace(['-'], 0)
df['RSSI'] = df['RSSI'].replace(['-'], 0)

In [ ]:
df['State'] = df['State'].replace('D',1)
df['State'] = df['State'].replace('I',0)

In [ ]:
df["State"] = df["State"].apply(pd.to_numeric)


In [ ]:
df["ServingCell_Lon"] = df["ServingCell_Lon"].apply(pd.to_numeric)
df["ServingCell_Lat"] = df["ServingCell_Lat"].apply(pd.to_numeric)
df["RSRQ"] = df["RSRQ"].apply(pd.to_numeric)
df["RSSI"] = df["RSSI"].apply(pd.to_numeric)
df["CQI"] = df["CQI"].apply(pd.to_numeric)
df["SNR"] = df["SNR"].apply(pd.to_numeric)


In [ ]:
df['ServingCell_Lon'] = df['ServingCell_Lon'].replace(0,df["ServingCell_Lon"].mode()[0])
df['ServingCell_Lat'] = df['ServingCell_Lat'].replace(0,df["ServingCell_Lat"].mode()[0])
df['RSRQ'] = df['RSRQ'].replace(0,df["RSRQ"].mode()[0])
df['SNR'] = df['SNR'].replace(0,df["SNR"].mode()[0])
df['CQI'] = df['CQI'].replace(0,df["CQI"].mode()[0])
df['RSSI'] = df['RSSI'].replace(0,df["RSSI"].mode()[0])

In [ ]:
df["RSRP"] = df["RSRP"].apply(pd.to_numeric)
df["Longitude"] = df["Longitude"] .apply(pd.to_numeric)
df["Latitude"] = df["Latitude"] .apply(pd.to_numeric)
df["Speed"] = df["Speed"].apply(pd.to_numeric)
df["CellID"] = df["CellID"].apply(pd.to_numeric)
df["DL_bitrate"] = df["DL_bitrate"].apply(pd.to_numeric)
df["UL_bitrate"] = df["UL_bitrate"].apply(pd.to_numeric)



In [ ]:
df.dtypes

Longitude               float64
Latitude                float64
Speed                     int64
CellID                    int64
RSRP                      int64
RSRQ                      int64
SNR                     float64
CQI                       int64
RSSI                      int64
DL_bitrate                int64
UL_bitrate                int64
State                     int64
ServingCell_Lon         float64
ServingCell_Lat         float64
ServingCell_Distance    float64
dtype: object

In [ ]:
df.shape

(174524, 15)

In [ ]:
dups = df.duplicated()
df[dups]

,Longitude,Latitude,Speed,CellID,RSRP,RSRQ,SNR,CQI,RSSI,DL_bitrate,UL_bitrate,State,ServingCell_Lon,ServingCell_Lat,ServingCell_Distance
3,-8.499733,51.893346,0,2,-95,-13,13.0,9,-80,0,0,0,-8.491719,51.893905,553.43
5,-8.499887,51.893384,1,2,-95,-13,13.0,9,-80,0,0,0,-8.491719,51.893905,563.48
160,-8.497641,51.894025,0,2,-94,-13,12.0,9,-77,5826,110,1,-8.491719,51.893905,406.59
265,-8.495939,51.894389,3,2,-98,-13,6.0,6,-83,3008,56,1,-8.491719,51.893905,294.54
492,-8.492823,51.895066,1,2,-101,-15,0.0,5,-85,1308,33,1,-8.491719,51.893905,149.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173044,-8.490956,51.934883,74,34317,-101,-2,0.0,0,0,0,0,0,0.000000,0.000000,0.00
173046,-8.490956,51.934883,74,34324,-101,-2,0.0,0,0,0,0,0,-8.513451,51.876173,6708.18
173082,-8.490956,51.934883,74,1,-100,-12,12.0,11,0,43103,756,1,-8.489758,51.928426,722.67
173848,-8.484568,51.896165,2,3,-72,-12,17.0,11,-55,45328,797,1,-8.483589,51.894878,158.09


In [ ]:
df.drop_duplicates(keep=False, inplace=True)

In [ ]:
df.shape

(158916, 15)

In [ ]:
'''       SAVE TIME

import plotly.express as px

for col in df.columns:
  fig=px.box(df,y=col)
  fig.show()

  '''

'       SAVE TIME\n\nimport plotly.express as px\n\nfor col in df.columns:\n  fig=px.box(df,y=col)\n  fig.show()\n\n  '

In [ ]:
for col in df.columns:                        #calc z score and number of outliers ---- not many outliers so we ignore.
  if df[col].dtypes!=object:
    print(col)
    u=df[col].mean() + 3*df[col].std()
    l=df[col].mean() - 3*df[col].std()
    print("Upper limit",u)
    print("Lower limit",l)
    print(len(df[(df[col] > u) | (df[col]< l)]))

Longitude
Upper limit -7.3861366489427205
Lower limit -9.563384929186604
4206
Latitude
Upper limit 52.734919174495815
Lower limit 51.224034848082155
6684
Speed
Upper limit 128.29178255217266
Lower limit -76.01367336241202
3966
CellID
Upper limit 35350.26712445113
Lower limit -28278.77311503735
5792
RSRP
Upper limit -44.60475283545407
Lower limit -138.70114461980532
16
RSRQ
Upper limit 5.910954316761451
Lower limit -25.715920462398138
2
SNR
Upper limit 25.477544199777896
Lower limit -17.624879899141078
709
CQI
Upper limit 19.175770312734787
Lower limit -7.219541865001394
0
RSSI
Upper limit 62.900686009919646
Lower limit -169.11158988366427
0
DL_bitrate
Upper limit 55034.143350255305
Lower limit -31541.367053343733
2949
UL_bitrate
Upper limit 871.3636887356301
Lower limit -470.6235744614223
2681
State
Upper limit 1.4605308926584981
Lower limit 0.4851385175959758
4317
ServingCell_Lon
Upper limit 4.813948484574485
Lower limit -17.542328452066744
0
ServingCell_Lat
Upper limit 106.5865736894

In [ ]:
df.columns

Index(['Longitude', 'Latitude', 'Speed', 'CellID', 'RSRP', 'RSRQ', 'SNR',
       'CQI', 'RSSI', 'DL_bitrate', 'UL_bitrate', 'State', 'ServingCell_Lon',
       'ServingCell_Lat', 'ServingCell_Distance'],
      dtype='object')

In [ ]:
#splitting x and y
X=df.iloc[:,:]
X=X.drop(['State'],axis=1)
X.shape

(158916, 14)

In [ ]:
y=df['State']
y.shape

(158916,)

In [ ]:
y.head()

0    1
1    0
6    0
7    1
8    1
Name: State, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print('X_train : ')
print(X_train.head())
print('')
print('X_test : ')
print(X_test.head())
print('')
print('y_train : ')
print(y_train.head())
print('')
print('y_test : ')
print(y_test.head())

X_train : 
       Longitude   Latitude  Speed  CellID  RSRP  RSRQ  SNR  CQI  RSSI  \
81309  -8.485373  51.897197      0    5306   -63    -2  0.0    0     0   
76729  -8.462424  51.896592     35   40450   -65    -2  0.0    0     0   
33484  -8.501377  51.892998      1       2  -108   -15  0.0    8   -91   
8125   -8.501781  51.893346      2       8  -109   -15  8.0    8   -92   
49016  -8.495834  51.894385      2       2  -103   -14  0.0    7   -86   

       DL_bitrate  UL_bitrate  ServingCell_Lon  ServingCell_Lat  \
81309        6363         118         0.000000         0.000000   
76729       13868         253         0.000000         0.000000   
33484         548          14        -8.491719        51.893905   
8125         9528         175         0.000000         0.000000   
49016       14382         266        -8.491719        51.893905   

       ServingCell_Distance  
81309                  0.00  
76729                  0.00  
33484                670.37  
8125                 

In [ ]:
print('X_train : ')
print(X_train.shape)
print('')
print('X_test : ')
print(X_test.shape)
print('')
print('y_train : ')
print(y_train.shape)
print('')
print('y_test : ')
print(y_test.shape)

X_train : 
(111241, 14)

X_test : 
(47675, 14)

y_train : 
(111241,)

y_test : 
(47675,)


In [ ]:
#RANDOM FOREST CLASSIFIER

from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification


In [ ]:
rf = RandomForestClassifier(random_state=0, n_estimators=500)
rf.fit(X, y)
print(rf.predict(X_test))

[1 1 1 ... 1 1 1]


In [ ]:
(rf.predict(X_test) == 0).sum().sum()

1306

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import math
# Given values
Y_true = y_test  # Y_true = Y (original values)

# calculated values
y_pred = rf.predict(X_test)  # Y_pred = Y'

# Calculation of Mean Squared Error (MSE)
mse=mean_squared_error(Y_true,y_pred)
rmse = math.sqrt(mse)
print(mse)
print("The difference between actual and predicted values", rmse)
print(mean_absolute_error(y_test, y_pred))

0.0005873099108547456
The difference between actual and predicted values 0.024234477730183203
0.0005873099108547456


In [ ]:
#K MEANS CLUSTERING

In [ ]:
X1=df.iloc[:,:]
y1=df['State']

In [ ]:
X1.columns

Index(['Longitude', 'Latitude', 'Speed', 'CellID', 'RSRP', 'RSRQ', 'SNR',
       'CQI', 'RSSI', 'DL_bitrate', 'UL_bitrate', 'State', 'ServingCell_Lon',
       'ServingCell_Lat', 'ServingCell_Distance'],
      dtype='object')

In [ ]:
cols = X1.columns

from sklearn.preprocessing import MinMaxScaler

ms = MinMaxScaler()

X1 = ms.fit_transform(X1)

X1 = pd.DataFrame(X1, columns=[cols])

X1.head()



,Longitude,Latitude,Speed,CellID,RSRP,RSRQ,SNR,CQI,RSSI,DL_bitrate,UL_bitrate,State,ServingCell_Lon,ServingCell_Lat,ServingCell_Distance
0,0.330952,0.014876,0.000000,0.000031,0.606936,0.354839,0.548387,0.666667,0.148936,0.000000,0.000000,1.0,0.180582,0.967922,0.002475
1,0.330952,0.014876,0.000000,0.000031,0.606936,0.354839,0.516129,0.533333,0.170213,0.000000,0.000000,0.0,0.180582,0.967922,0.002475
2,0.330896,0.014909,0.006024,0.000031,0.595376,0.354839,0.451613,0.600000,0.148936,0.000000,0.000000,0.0,0.180582,0.967922,0.002530
3,0.330896,0.014909,0.006024,0.000031,0.595376,0.354839,0.451613,0.600000,0.148936,0.012421,0.019866,1.0,0.180582,0.967922,0.002530
4,0.330973,0.014976,0.000000,0.000031,0.595376,0.354839,0.451613,0.600000,0.148936,0.012421,0.019866,1.0,0.180582,0.967922,0.002447


In [ ]:


from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=4, random_state=0)

kmeans.fit(X1)




/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=4, random_state=0)

In [ ]:


kmeans.cluster_centers_



array([[ 3.29550934e-01,  3.59666523e-02,  1.07090460e-01,
         3.07473228e-05,  5.95356991e-01,  3.66261452e-01,
         5.74526368e-01,  5.49008044e-01,  1.55646587e-01,
         8.67472960e-02,  5.97827975e-02,  9.78342693e-01,
         1.75786908e-01,  9.68529884e-01,  2.13532681e-02],
       [ 3.59581203e-01,  1.65528900e-01,  2.85679874e-01,
         1.89190139e-01,  7.19507273e-01,  6.97532133e-01,
         4.85414812e-01,  1.09459778e-02,  1.00000000e+00,
         2.74342087e-02,  2.30256928e-02,  9.66560510e-01,
         1.00000000e+00,  2.20601315e-13,  8.77770079e-16],
       [ 3.35560668e-01,  9.55126204e-02,  2.08888159e-01,
         1.13658572e-01,  6.44034291e-01,  5.29068622e-01,
         5.14008630e-01,  2.61148513e-01,  9.99871747e-01,
         4.83361526e-02,  3.56591868e-02,  9.71765565e-01,
         1.80907505e-01,  9.70317838e-01,  3.82938593e-02],
       [ 3.74686212e-01,  9.68877213e-02,  1.06831216e-01,
         1.60029245e-04,  5.60129618e-01,  3.21489937

In [ ]:
kmeans.inertia_

33056.67170583633

In [ ]:
labels = kmeans.labels_

# check how many of the samples were correctly labeled
correct_labels = sum(y1 == labels)

print("Result: %d out of %d samples were correctly labeled." % (correct_labels, y1.size))


print('Accuracy score: {0:0.2f}'. format(correct_labels/float(y1.size)))




Result: 34930 out of 158916 samples were correctly labeled.
Accuracy score: 0.22


In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5,random_state=0)

kmeans.fit(X1)

labels = kmeans.labels_

# check how many of the samples were correctly labeled


correct_labels = sum(y1 == labels)
print("Result: %d out of %d samples were correctly labeled." % (correct_labels, y1.size))

print('Accuracy score: {0:0.2f}'. format(correct_labels/float(y1.size)))



/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Result: 34447 out of 158916 samples were correctly labeled.
Accuracy score: 0.22
